In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("./US_HSV_modes_top60_cities_weather_user.csv")

/Library/Python/2.7/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7588558 entries, 0 to 7588557
Data columns (total 28 columns):
image.path    object
filename      object
img_url       object
size_h        int64
size_w        int64
lat           float64
lon           float64
postedTime    object
mongo_id      object
location      object
R.mode        int64
G.mode        int64
B.mode        int64
R.entropy     float64
G.entropy     float64
B.entropy     float64
city          object
H.mode        int64
S.mode        int64
V.mode        int64
hour          int64
local_date    object
temp          float64
humidity      object
wind          float64
rain          float64
sky           float64
actor         int64
dtypes: float64(9), int64(10), object(9)
memory usage: 1.6+ GB


In [4]:
df = df[['filename','lat','lon','city']]

In [6]:
df = df[df.city=="san diego"]

In [15]:
df.reset_index(drop=True,inplace=True)

In [9]:
from xml.etree import ElementTree
import keytree
import shapely
from shapely.geometry import Polygon, Point, shape

In [10]:
doc = open("/Users/damoncrockett/vosd.org/215-opendsd/Final.kml").read()
tree = ElementTree.fromstring(doc)
# kml namespace
kmlns = tree.tag.split('}')[0][1:]
# find all placemarks
placemks = tree.findall(".//{%s}Placemark" % kmlns)
# filter out those without polygon elements
placemks_with_polygons = []

for p in placemks:
    if p.findall(".//{%s}Polygon" % kmlns):
        placemks_with_polygons.append(p)

In [11]:
# extract kml LinearRings, convert to shapely LinearRings, make dict with cpa names

def coords_names(placemks):
  coords_names_dict = {}
  for placemk in placemks:
    name = placemk.getchildren()[0].text
    coord_text = placemk.findtext(".//{%s}coordinates" % kmlns)
    coords = []
    for elems in coord_text.split():
      points = elems.split(",")
      coords.append((float(points[0]), float(points[1])))
    coords_names_dict[Polygon(coords)] = name
  return coords_names_dict

import pandas as pd

cpa_polygons = pd.DataFrame(coords_names(placemks_with_polygons).items(),
            columns=['Polygon','CPA'])

In [16]:
# collect all lat lon points from dataframe

n = len(df.index)
locations = []
for i in range(n):
    point = Point(df.lon[i],df.lat[i])
    locations.append(point)

In [17]:
# crucial step: build spatial index

from rtree import index
idx = index.Index()
count = -1
for item in cpa_polygons.Polygon:
    count +=1
    idx.insert(count, item.bounds)

In [18]:
# assign a cpa to each point

m = len(locations)
hoods = []
for i in range(m):
    tmp = 'nan'
    for j in idx.intersection((df.lon[i],df.lat[i])):
        if locations[i].within(cpa_polygons.Polygon.loc[j]):
            tmp = cpa_polygons.CPA[j]
            break
    hoods.append(tmp)
    
df['CPA'] = hoods

In [20]:
df.CPA.value_counts()

nan                                    74349
DOWNTOWN                               18807
COLLEGE AREA                            8476
MISSION VALLEY                          7442
UNIVERSITY                              7250
PENINSULA                               5650
MIRA MESA                               5529
PACIFIC BEACH                           5387
SKYLINE-PARADISE HILLS                  5320
CLAIREMONT MESA                         4857
OTAY MESA-NESTOR                        4428
NAVAJO                                  4240
UPTOWN                                  3957
OCEAN BEACH                             3885
MID-CITY:CITY HEIGHTS                   3808
LINDA VISTA                             3742
GREATER NORTH PARK                      3670
LA JOLLA                                3578
ENCANTO NEIGHBORHOODS,SOUTHEASTERN      3522
TIERRASANTA                             3236
KEARNY MESA                             3081
SOUTHEASTERN SAN DIEGO,SOUTHEASTERN     2916
RANCHO PEN

In [21]:
df = df[df.CPA!="nan"]

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 151434 entries, 3 to 225782
Data columns (total 5 columns):
filename    151434 non-null object
lat         151434 non-null float64
lon         151434 non-null float64
city        151434 non-null object
CPA         151434 non-null object
dtypes: float64(2), object(3)
memory usage: 6.9+ MB


In [23]:
df.head()

,filename,lat,lon,city,CPA
3,A_zWavdCEAA5R-p.jpg,32.548669,-117.053793,san diego,SAN YSIDRO
4,BAI1MoECYAA_Svp.jpg,32.729007,-117.190861,san diego,RESERVE
5,BALG7R7CEAI8z0P.jpg,32.753788,-117.103402,san diego,MID-CITY:CITY HEIGHTS
9,BA5dTyxCEAApln6.jpg,32.714065,-117.022723,san diego,SKYLINE-PARADISE HILLS
10,BBEhK_rCMAErrnS.jpg,32.693186,-117.067500,san diego,"ENCANTO NEIGHBORHOODS,SOUTHEASTERN"


In [24]:
df.to_csv("./sd_2013.csv",index=False)